# Data Base connectivity

## JDBC

> Java Database Connectivity (JDBC) is an application programming interface (API) for the Java programming language which defines how a client may access a database. It is a Java-based data access technology used for Java database connectivity. It is part of the Java Standard Edition platform, from Oracle Corporation. It provides methods to query and update data in a database, and is oriented toward relational databases. A JDBC-to-ODBC bridge enables connections to any ODBC-accessible data source in the Java virtual machine (JVM) host environment.

https://en.wikipedia.org/wiki/Java_Database_Connectivity

# How JDBC Works
https://www.baeldung.com/java-jdbc

## JDBC Drivers

The most commonly used type is JDBC 4: 
connect directly to a database by converting JDBC calls into database-specific calls; known as database protocol drivers or thin drivers.

:::: {.columns}

::: {.fragment .column width="50%"}
**Advantages**: platform-independent. Connecting directly to a database server provides better performance compared to other types.
::: 

::: {.fragment .column width="50%"}
**Disadvantages**: Database-specific – given each database has its own specific protocol.
:::
::::


## MySql connector 
MySql develops its connector (not only Java)

<https://www.mysql.com/products/connector/>


## Adding API to a Maven Project

<https://mvnrepository.com/artifact/com.mysql/mysql-connector-j/8.2.0>

```xml
<dependency>
    <groupId>com.mysql</groupId>
    <artifactId>mysql-connector-j</artifactId>
    <version>8.2.0</version>
</dependency>
```

## Driver Registration

As of JDBC 4.0, all drivers that are found in the classpath are automatically loaded. 

Therefore, we won’t need this Class.forName part in modern environments.

Before
```java
Class.forName("com.mysql.cj.jdbc.Driver")
``` 

## Create connection

To open a connection, we can use the getConnection() method of DriverManager class. 

This method requires a connection URL String parameter

Mysql allows to specify credentials as parameters, or passed ad parameter in getConnection 

[Java doc](https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/DriverManager.html#getConnection(java.lang.String,java.lang.String,java.lang.String))

### Example of JDBC connection in a Servlet
```java

import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.SQLException;

...
    
    Connection connnection; 
    final String connectionString = "jdbc:mysql://localhost:3306/students?user=root&password=ruttolibero";

    public void init() {
        System.out.println("Servlet " + this.getServletName() + " has started");
        try {
             connnection = DriverManager.getConnection(connectionString);
        } catch (SQLException e) {
            // This print goes into catalina.out or standard output with Jetty
            System.out.println("Error while connecting to database");
            System.out.println("SQLException: " + e.getMessage());
            System.out.println("SQLState: " + e.getSQLState());
            System.out.println("VendorError: " + e.getErrorCode());
            return;
        }
        System.out.println("Connection Successfull");
    }


```

## Query the Database - 1. Statement
<https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/Statement.html>

> Stament is the object used for executing a static SQL statement and returning the results it produces.

::::{.columns}

::: {.fragment .column width="50%"}
* Statement are created using Connection
* Executed using 
    - executeQuery() for SELECT instructions
    - executeUpdate() for updating the data or the database structure
    - execute() can be used for both cases above when the result is unknown
:::

::: {.fragment .column width="50%"}
- For DML queries: getUpdateCount() method returns the number of rows affected.
- Select: getResultSet() returns a [Resultset](https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/ResultSet.html)
:::

::::


## Example: add in Hello World Servlet a query to select all students
```java

// To query the database
import java.sql.Statement;
import java.sql.ResultSet;

try {
            Statement stmt = connnection.createStatement();
            ResultSet result = stmt.executeQuery("select * from student");
            while (result.next()) {
                out.write("<p>" + result.getString("name") + "</p>");
            }
        } catch (SQLException e) {
            System.out.println("Error while querying the database");
            System.out.println("SQLException: " + e.getMessage());
            System.out.println("SQLState: " + e.getSQLState());
            System.out.println("VendorError: " + e.getErrorCode());
        }
```

## Query the Database - 2. PreparedStatement
https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/PreparedStatement.html

> A SQL statement is precompiled and stored in a PreparedStatement object. This object can then be used to efficiently execute this statement multiple times.

::::{.columns}

::: {.fragment .column width="50%"}
* PreparedStatement are created using Connection
* Parameters are marked using a ?
* Setter methods allow to add parameters
* Executed using same methods (without SQL)
    - executeQuery() for SELECT instructions
    - executeUpdate() for updating the data or the database structure
    - execute() can be used for both cases above when the result is unknown
:::

::: {.fragment .column width="50%"}
- DML queries:executeUpdate() method returns the number of rows affected.
- Select: getResultSet() returns a [Resulset](https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/ResultSet.html)
:::

::::


## Example: Query to insert a student

```java

public void doPost(HttpServletRequest request, HttpServletResponse response) throws IOException {
        // Need to check parameters
        String name=request.getParameter("name");
        Integer age=Integer.parseInt(request.getParameter("age"));
        String query="INSERT INTO student (name, age) VALUES (?, ?)";
        try {
            java.sql.PreparedStatement stmt = connnection.prepareStatement(query);
            stmt.setString(1, name);
            stmt.setInt(2, age);
            int rowsAffected =stmt.executeUpdate();
            System.out.println("Rows affected: " + rowsAffected);
        } catch (SQLException e) {
            System.out.println("Error while insert into the database");
            System.out.println("SQLException: " + e.getMessage());
            System.out.println("SQLState: " + e.getSQLState());
            System.out.println("VendorError: " + e.getErrorCode()); 
        }
    }
```

## Test 

Visual Studio Extension
https://marketplace.visualstudio.com/items?itemName=humao.rest-client

```
POST http://localhost:9999/hello HTTP/1.1
content-type: application/x-www-form-urlencoded

name=Nics&age=5
```

## Getting Results - ResultSets
https://docs.oracle.com/en/java/javase/21/docs/api/java.sql/java/sql/ResultSet.html

> A table of data representing a database result set, which is usually generated by executing a statement that queries the database.

> A ResultSet object maintains a cursor pointing to its current row of data. Initially the cursor is positioned before the first row. The next method moves the cursor to the next row, and because it returns false when there are no more rows in the ResultSet object, it can be used in a while loop to iterate through the result set.

> A default ResultSet object is not updatable and has a cursor that moves forward only. Thus, you can iterate through it only once and only from the first row to the last row. It is possible to produce ResultSet objects that are scrollable and/or updatable. The following code fragment, in which con is a valid Connection object, illustrates how to make a result set that is scrollable and insensitive to updates by others, and that is updatable. See ResultSet fields for other options.

## Out of Scope
- Updatable Result Sets
- Parsing Meta Data
- Result Set Meta Data
- Transactions

## Closing the Resources

https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/AutoCloseable.html

When we’re no longer using it, we need to close the connection to release database resources.

Same for Statement* and ResultSets.

We can do this using the close() API:

```java
statement.close();
rs.close();
connection.close();
```

# Jakarta Persistence
https://jakarta.ee/specifications/persistence/3.1/jakarta-persistence-spec-3.1.html


# Scope
Jakarta Persistence defines a standard for management of persistence and object/relational mapping in Java® environments.

A class
```java
@Entity
public class Customer implements Serializable {
    private Long id;
    private String name;
    private Address address;
    private Collection<Order> orders = new HashSet();
    private Set<PhoneNumber> phones = new HashSet();

    // No-arg constructor
    public Customer() {}

    @Id // property access is used
    public Long getId() {
        return id;
    }

    public void setId(Long id) {
        this.id = id;
    }

    public String getName() {
        return name;
    }

    public void setName(String name) {
        this.name = name;
    }

    public Address getAddress() {
        return address;
    }

    public void setAddress(Address address) {
        this.address = address;
    }

    @OneToMany
    public Collection<Order> getOrders() {
        return orders;
    }

    public void setOrders(Collection<Order> orders) {
        this.orders = orders;
    }

    @ManyToMany
    public Set<PhoneNumber> getPhones() {
        return phones;
    }

    public void setPhones(Set<PhoneNumber> phones) {
        this.phones = phones;
    }

    // Business method to add a phone number to the customer
    public void addPhone(PhoneNumber phone) {
        this.getPhones().add(phone);

        // Update the phone entity instance to refer to this customer
        phone.addCustomer(this);
    }
}
```



![](images/JPA-DB.jpg)

# Creating a Servlet with JPA (the hard way)

Create a Web App to manage students using a single servlet to get the list of student, using post create a new student. 

Use a JSP to send data using and old fashion HTML form

# Step 0
Create a maven project as we did before adding Jetty, Servlet

# Step 1
Add dependecies to pom.xml
```xml
<dependency>
    <groupId>org.hibernate.orm</groupId>
    <artifactId>hibernate-core</artifactId>
    <version>6.4.0.Final</version>
</dependency>
<dependency>
    <groupId>jakarta.persistence</groupId>
    <artifactId>jakarta.persistence-api</artifactId>
    <version>3.1.0</version>
</dependency>
<dependency>
    <groupId>com.mysql</groupId>
    <artifactId>mysql-connector-j</artifactId>
    <version>8.2.0</version>
</dependency>
```



# Step 2
create persistence.xml file under src/main/resources/META-INF

Please note that class name can be different 
```xml
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<persistence version="3.0" xmlns="https://jakarta.ee/xml/ns/persistence"
             xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
             xsi:schemaLocation="https://jakarta.ee/xml/ns/persistence https://jakarta.ee/xml/ns/persistence/persistence_3_0.xsd">
    <persistence-unit name="default">
        <class>edu.unict.tsdw.lesson</class>
        <properties>
            <property name="jakarta.persistence.jdbc.driver" value="com.mysql.cj.jdbc.Driver"/>
            <property name="jakarta.persistence.jdbc.url" value="jdbc:mysql://localhost/students"/>
            <property name="jakarta.persistence.jdbc.user" value="root"/>
            <property name="jakarta.persistence.jdbc.password" value="ruttolibero"/>
           <!-- <property name="hibernate.hbm2ddl.auto" value="update"/> -->
        </properties>
    </persistence-unit>
</persistence>
```



# Step 3.1
create Student class 
```java
package edu.unict.tsdw.lesson;

import jakarta.persistence.Entity;
import jakarta.persistence.GeneratedValue;
import jakarta.persistence.Id;

@Entity
public class Student {
    @Id
    @GeneratedValue
    private Long id;
    private String name;
    private Integer age;

    public void setId(Long id) {
        this.id = id;
    }

    public Long getId() {
        return id;
    }

    public String getName() {
        return name;
    }

    public void setName(String name) {
        this.name = name;
    }

    public Integer getAge() {
        return age;
    }

    public void setAge(Integer age) {
        this.age = age;
    }

    @Override
    public String toString() {
        return "Student{" +
                "name='" + name + '\'' +
                ", id='" + id + '\'' + 
                ", age='" + age + '\'' +
                '}';
    }
}
```



# Step 3.2
create newstudent.jsp
```jsp
<%@ page contentType="text/html; charset=UTF-8" pageEncoding="UTF-8" %>
<!DOCTYPE html>
<html>
<head>
    <title>New student</title>
</head>
<body>
<form method="post" action="hello" autocomplete="off">
    <label for="name">What's your name?</label><br>
    <input type="text" id="name" name="name"><br>
    <input type="submit" value="Submit">
</form>
</body>
</html>
```



# Step 3.3
Implement doPost with JPA
```java
    in HelloWorld.java (extension will add automatically)
    import jakarta.persistence.EntityManager;
    import jakarta.persistence.EntityManagerFactory;
    import jakarta.persistence.EntityTransaction;
    import jakarta.persistence.Persistence;
    
    in doPost..
    // Add a class attribute
    EntityManager em;
    // in init
    EntityManagerFactory emf = Persistence.createEntityManagerFactory("default");
    em = emf.createEntityManager();
    
    // in doPost
    Student student=new Student();
    student.setName(name);
    student.setAge(age);
    EntityTransaction transaction = em.getTransaction();
    transaction.begin();
    em.persist(student);
    transaction.commit();
    }
```



# Step 3.4
Implement doGet with JPA
```java
  List<Student> Student = em
        .createQuery("Select m from Student m ", Student.class)
        .getResultList(); 
        for (Student s : Student) {
            out.write("<p>Student " + s.getName() + " Age "+s.getAge()+"</p>");
        }
    }
```



# A bit of theory

https://jakarta.ee/specifications/persistence/3.1/apidocs/jakarta.persistence/module-summary.html
https://docs.jboss.org/hibernate/orm/6.1/userguide/html_single/Hibernate_User_Guide.html


## Entity 
https://medium.com/@saeiddrv/jpa-hibernate-mapping-types-891686bc6cfd

An entity is a domain model class that we want to map as a table in a relational database. Thus, each instance of the entity class corresponds to a specific row in the mapped table.

![](https://miro.medium.com/v2/resize:fit:460/format:webp/1*R3oYMoR_UZrzLJg-blLbbg.png)

## Entity Annotation 

Defined in https://jakarta.ee/specifications/persistence/3.1/apidocs/jakarta.persistence/jakarta/persistence/package-summary

Most popular:

| Annotation | Description |
| -----------|------------|
| Entity     | Specifies that the class is an entity. |
| Id         | Specifies the primary key of an entity.|
| Column     | Specifies the mapped column for a persistent property or field. |
| OneToMany  | Specifies a many-valued association with one-to-many multiplicity. (same for other relations)|

## Queries
https://www.baeldung.com/jpa-queries

## Query

Java Persistence Query Language (JPQL) syntax is similar to SQL

```sql
SELECT ... FROM ...
[WHERE ...]
[GROUP BY ... [HAVING ...]]
[ORDER BY ...]
```

They generally used to perform CRUD operations.

Named parameters are managed using :params



### TypedQuery

Like Query but specyfing the return class of the query to achieve **stronged types** without cast

### Example
```java

import javax.persistence.EntityManager;
import javax.persistence.TypedQuery;

public List<Author> getAuthorsByLastName(String lastName) {
    String queryString = "SELECT a FROM Author a " +
                         "WHERE a.lastName IS NULL OR LOWER(a.lastName) = LOWER(:lastName)";

    TypedQuery<Author> query = getEntityManager().createQuery(queryString, Author.class);
    query.setParameter("lastName", lastName);
    return query.getResultList();
}
```

## NativeQuery

A NativeQuery is simply an SQL query. These allow us to unleash the full power of our database, as we can use proprietary features not available in JPQL-restricted syntax.

### Example
```java
public UserEntity getUserByIdWithNativeQuery(Long id) {
    Query nativeQuery
      = getEntityManager().createNativeQuery("SELECT * FROM users WHERE id=:userId", UserEntity.class);
    nativeQuery.setParameter("userId", id);
    return (UserEntity) nativeQuery.getSingleResult();
}
```

# An example with Docker
https://github.com/unict-dmi-tswd-lab/myWebApp-servletdemo

## References
- https://maven.apache.org/archetypes/maven-archetype-webapp/
- https://dev.to/hantsy_26/building-your-own-maven-archetype-gm8
- https://itnext.io/building-jakarta-ee-9-web-application-with-servlet-containers-b3acc50c8464
- https://start.jakarta.ee/
- https://www.javatpoint.com/steps-to-create-a-servlet-using-tomcat-server
- https://www.jetbrains.com/help/idea/creating-and-running-your-first-java-ee-application.html#new_project
- https://www.baeldung.com/tomcat-deploy-war
- https://www.eclipse.org/jetty/documentation/jetty-11/programming-guide/index.html#jetty-maven-plugin
- https://itnext.io/building-jakarta-ee-9-web-application-with-servlet-containers-b3acc50c8464
- https://github.com/Azure-Samples/tomcat10-jakartaee9
- https://blog.payara.fish/getting-started-with-jakarta-ee-9-jakarta-persistence-api-jpa
- https://itnext.io/whats-new-in-jakarta-persistence-3-1-by-examples-81b292e8b3a4
- https://maven.apache.org/archetypes/maven-archetype-webapp/
- https://itnext.io/generate-a-jakarta-ee-9-project-skeleton-from-maven-archetype-2b9fc9ce9bb8 https://github.com/hantsy/maven-archetype-jakartaee9
- https://start.jakarta.ee/
- https://blog.payara.fish/getting-started-with-jakarta-ee-9-hello-world